In [1]:
# Importing Libraries

In [152]:
import pandas as pd
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
import pickle
from sklearn.metrics import accuracy_score

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [9]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [11]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [12]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [13]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Users\mishra_s\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Users\mishra_s\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [19]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [20]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [23]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 92s 13ms/step - loss: 1.3018 - acc: 0.4395 - val_loss: 1.1254 - val_acc: 0.4662
Epoch 2/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.9666 - acc: 0.5880 - val_loss: 0.9491 - val_acc: 0.5714
Epoch 3/30
7352/7352 [==============================] - 97s 13ms/step - loss: 0.7812 - acc: 0.6408 - val_loss: 0.8286 - val_acc: 0.5850
Epoch 4/30
7352/7352 [==============================] - 95s 13ms/step - loss: 0.6941 - acc: 0.6574 - val_loss: 0.7297 - val_acc: 0.6128
Epoch 5/30
7352/7352 [==============================] - 92s 13ms/step - loss: 0.6336 - acc: 0.6912 - val_loss: 0.7359 - val_acc: 0.6787
Epoch 6/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.5859 - acc: 0.7134 - val_loss: 0.7015 - val_acc: 0.6939
Epoch 7/30
7352/7352 [==============================] - 95s 13ms/step - loss: 0.5692 - acc: 0.7477 - val_loss: 0.5995 - val_acc: 0.7387


In [24]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 512        0        25        0                   0   
SITTING                  3      410        75        0                   0   
STANDING                 0       87       445        0                   0   
WALKING                  0        0         0      481                   2   
WALKING_DOWNSTAIRS       0        0         0        0                 382   
WALKING_UPSTAIRS         0        0         0        2                  18   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            3  
STANDING                           0  
WALKING                           13  
WALKING_DOWNSTAIRS                38  
WALKING_UPSTAIRS                 451  


In [27]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 4s 2ms/step


In [28]:
score

[0.3087582236972612, 0.9097387173396675]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

## LSTM models with 9 hidden units

In [21]:
n_hidden = 9

In [22]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 9)                 684       
_________________________________________________________________
dropout_2 (Dropout)          (None, 9)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 60        
Total params: 744
Trainable params: 744
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [24]:
# Training the model
model_result = model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.cast instead.
Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 50s 7ms/step - loss: 1.5873 - acc: 0.3575 - val_loss: 1.4696 - val_acc: 0.4479
Epoch 2/30
7352/7352 [==============================] - 48s 7ms/step - loss: 1.3778 - acc: 0.4276 - val_loss: 1.3392 - val_acc: 0.4791
Epoch 3/30
7352/7352 [==============================] - 46s 6ms/step - loss: 1.2766 - acc: 0.4518 - val_loss: 1.2409 - val_acc: 0.5127
Epoch 4/30
7352/7352 [==============================] - 46s 6ms/step - loss: 1.2307 - acc: 0.4689 - val_loss: 1.1858 - val_acc: 0.5151
Epoch 5/30
7352/7352 [==============================] - 46s 6ms/step - loss: 1.1720 - acc: 0.4874 - val_loss: 1.1393 - val_acc: 0.4890
Epoch 6/30
7352/7352 [==============================] - 48s 7ms/step - loss: 1.1577 - acc: 0.5126 - val_loss: 1.1051 - val_acc: 0.5735
Epoch 7/30
7352/7352 [==============================] - 48s 6ms/step - loss: 1.1198 - acc: 0.5

In [25]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         5        0                   0   
SITTING                  1      398        92        0                   0   
STANDING                 0      103       420        1                   0   
WALKING                  0        0        13      355                  39   
WALKING_DOWNSTAIRS       1        0         1      320                  86   
WALKING_UPSTAIRS         0        0         0       75                  18   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            22  
SITTING                            0  
STANDING                           8  
WALKING                           89  
WALKING_DOWNSTAIRS                12  
WALKING_UPSTAIRS                 378  


In [26]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 526us/step


In [27]:
score

[0.6751788828571267, 0.7285374957583983]

- with fewer number of hidden units and dropout of 0.5, getting accuracy of 72%

## Model having 32 hidden units and 0.3 dropout rate

In [98]:
n_hidden = 32


# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.3))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_23 (LSTM)               (None, 32)                5376      
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [99]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [100]:
# Training the model
model_result = model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 67s 9ms/step - loss: 1.2652 - acc: 0.4523 - val_loss: 1.1144 - val_acc: 0.4717
Epoch 2/30
7352/7352 [==============================] - 57s 8ms/step - loss: 0.8755 - acc: 0.5983 - val_loss: 0.8536 - val_acc: 0.6563
Epoch 3/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.7634 - acc: 0.6508 - val_loss: 0.7893 - val_acc: 0.6010
Epoch 4/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.6754 - acc: 0.6587 - val_loss: 0.7699 - val_acc: 0.6196
Epoch 5/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.6008 - acc: 0.6861 - val_loss: 0.6942 - val_acc: 0.6895
Epoch 6/30
7352/7352 [==============================] - 62s 8ms/step - loss: 0.5590 - acc: 0.7365 - val_loss: 0.5912 - val_acc: 0.7360
Epoch 7/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.4780 - acc: 0.7712 - val_loss: 0.8360 - val_acc: 0.6861
Epoch 8

In [101]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      412        76        0                   2   
STANDING                 0      111       420        0                   0   
WALKING                  0        0         0      446                  13   
WALKING_DOWNSTAIRS       0        0         0        3                 347   
WALKING_UPSTAIRS         0        1         0        1                  15   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            1  
STANDING                           1  
WALKING                           37  
WALKING_DOWNSTAIRS                70  
WALKING_UPSTAIRS                 454  


In [102]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 2s 606us/step


In [103]:
score

[0.4676143296350844, 0.8876823888700374]

## Model with 2 LSTM layers

In [111]:
n_hidden = 20

In [112]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim), return_sequences=True))
model.add(LSTM(n_hidden))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 128, 20)           2400      
_________________________________________________________________
lstm_27 (LSTM)               (None, 20)                3280      
_________________________________________________________________
dropout_16 (Dropout)         (None, 20)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 126       
Total params: 5,806
Trainable params: 5,806
Non-trainable params: 0
_________________________________________________________________


In [113]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [114]:
# Training the model
model_result = model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=40)

Train on 7352 samples, validate on 2947 samples
Epoch 1/40
7352/7352 [==============================] - 107s 15ms/step - loss: 1.3844 - acc: 0.4117 - val_loss: 1.1680 - val_acc: 0.5158
Epoch 2/40
7352/7352 [==============================] - 100s 14ms/step - loss: 1.0511 - acc: 0.5297 - val_loss: 0.9515 - val_acc: 0.5877
Epoch 3/40
7352/7352 [==============================] - 100s 14ms/step - loss: 0.9090 - acc: 0.5698 - val_loss: 0.8279 - val_acc: 0.6105
Epoch 4/40
7352/7352 [==============================] - 99s 13ms/step - loss: 0.7957 - acc: 0.6147 - val_loss: 0.8115 - val_acc: 0.5833
Epoch 5/40
7352/7352 [==============================] - 99s 13ms/step - loss: 0.7439 - acc: 0.6412 - val_loss: 0.7850 - val_acc: 0.6213
Epoch 6/40
7352/7352 [==============================] - 100s 14ms/step - loss: 0.7346 - acc: 0.6453 - val_loss: 0.7462 - val_acc: 0.6434
Epoch 7/40
7352/7352 [==============================] - 100s 14ms/step - loss: 0.7007 - acc: 0.6523 - val_loss: 0.7445 - val_acc: 0.

In [115]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        27        0                   0   
SITTING                  0      416        67        1                   0   
STANDING                 0       95       433        4                   0   
WALKING                  0        0         0      438                   0   
WALKING_DOWNSTAIRS       0        0         0        0                 367   
WALKING_UPSTAIRS         0        1         0        3                   2   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            7  
STANDING                           0  
WALKING                           58  
WALKING_DOWNSTAIRS                53  
WALKING_UPSTAIRS                 465  


In [117]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 4s 1ms/step


In [118]:
score

[0.39639775041275266, 0.8920936545639634]

# Using divide an conquer 1DCNN

Reference
https://app.dimensions.ai/details/publication/pub.1101891256#readcube-epdf

Classification is divided into 2 stages:
- First stage consist of decision tree classifier where it classifies wheter activity is static or dynamic.
- Second stage consist of 2 models(1DCNN based) which predicts following:
    - if first stage predicts static: then this stage predict whether it's LAYING, SITTING, STANDING
    - if first stage predicts dynamic: then this stage predict whether it's WALKING, UP_STAIRS, DONW_STAIRS

## getting data

In [127]:
train = pd.read_csv('UCI_HAR_dataset/csv_files/train.csv')
test = pd.read_csv('UCI_HAR_dataset/csv_files/test.csv')
print(train.shape, test.shape)

(7352, 564) (2947, 564)


In [134]:
# get X_train and y_train from csv files
X_train = train.drop(['subject', 'Activity', 'ActivityName'], axis=1)
y_train = train.Activity

# get X_test and y_test from test csv file
X_test = test.drop(['subject', 'Activity', 'ActivityName'], axis=1)
y_test = test.Activity

### Loading Saved Models

In [122]:
first_stage_model = pickle.load(open('trained_models/first_stage_model.sav', 'rb'))

In [130]:
dynamic_model_path = "trained_models/dynamic_hra_model.h5"
dynamic_model = load_model(dynamic_model_path)

In [131]:
static_model_path = "trained_models/static_hra_model.h5"
static_model = load_model(static_model_path)

In [155]:
y_pred = []
for i in range(len(X_test)):
    first_stage_op = first_stage_model.predict([X_test.loc[i]])[0]
    if first_stage_op == 0: #it's dynamic activity
        predicted_values_prob = dynamic_model.predict(np.expand_dims([X_test.loc[i]], axis=2))
        predicted_value = np.argmax(predicted_values_prob[0])
        predicted_value += 1
        y_pred.append(predicted_value)
    else:
        predicted_values_prob = static_model.predict(np.expand_dims([X_test.loc[i]], axis=2))
        predicted_value = np.argmax(predicted_values_prob[0])
        predicted_value += 4
        y_pred.append(predicted_value)

In [156]:
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print(accuracy)

0.9636918900576857


In [161]:
Y_true = pd.Series([ACTIVITIES[y-1] for y in y_test])
Y_pred = pd.Series([ACTIVITIES[y-1] for y in y_pred])

print(pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred']))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      463        28        0                   0   
STANDING                 0       36       496        0                   0   
WALKING                  0        0         0      496                   0   
WALKING_DOWNSTAIRS       0        0         0        1                 416   
WALKING_UPSTAIRS         0        0         0       36                   3   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            0  
STANDING                           0  
WALKING                            0  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 432  


In [162]:
# Please compare all your models using Prettytable library
from prettytable import PrettyTable

table = PrettyTable();
table.field_names = ['Model', 'Accuracy'];
table.add_row(['LSTM Layers(2), Dropout(0.5), hidden_units(32)', '90'])
table.add_row(['LSTM Layers(1), Dropout(0.5), hidden_units(9)', '72.8'])
table.add_row(['LSTM Layers(1), Dropout(0.3), hidden_units(32)', '88.7'])
table.add_row(['LSTM Layers(2), Dropout(0.6), hidden_units(20)', '89.2'])
table.add_row(['Divide and conquer CNN', '96.36'])

print(table)

+------------------------------------------------+----------+
|                     Model                      | Accuracy |
+------------------------------------------------+----------+
| LSTM Layers(2), Dropout(0.5), hidden_units(32) |    90    |
| LSTM Layers(1), Dropout(0.5), hidden_units(9)  |   72.8   |
| LSTM Layers(1), Dropout(0.3), hidden_units(32) |   88.7   |
| LSTM Layers(2), Dropout(0.6), hidden_units(20) |   89.2   |
|             Divide and conquer CNN             |  96.36   |
+------------------------------------------------+----------+
